# Optimizing data access and processing

Over the multiple formats presented in the other tutorials, some were presented as cloud optimized. This notebook shows some methods to take advantage of those formats to optimize data access and processing. The methods presented are specifically optimized for cloud usage but can (and should most of the time) also be used with local files. The general idea is to limit the data access: either by loading only the required data or opening small fractions (chunks) of a whole dataset and iterate over all these chunks.

Some examples in this notebook use data downloaded or generated in other notebooks. Make sure you ran them before running this one. They also provide useful information and context on the files 
their formats.

### Differences between Raster data and Datacubes

Rasters are better suited than datacubes for representing single-layer data as they generally contains 2D data. Data cubes, on the other hand, handle multidimentional data and are more commonly used for analyzing multi-dimensional and lage scaled datasets. As an example, they are used in multispectral analysis and time-series analysis.

## Raster data

In the [raster formats tutorial](./raster_formats.ipynb), COGs (Cloud Optimized Geotiffs) were presented as the best cloud-oriented solution, as they are natively split and compressed in chunks and allow overviews. In this section, we present how to load and process a raster chunk by chunk. This is particularly usefull when using large datasets which can't be loaded in memory (both on local or cloud architectures) or when accessing a specific sub area of a temporal series of images.

### Using rasterio

Reading chunks with rasterio is done using a window: a smaller section of the image, defined by an offset and a size on the x and y axes.


A simple way to implement reading an image chunk by chunk is using chunks with the same width as the image (i.e. one or multiple lines) and iterating over them.

In [1]:
import rasterio
from rasterio.windows import Window
from utils import generate_hdf5_file

cog_path = "./sample_data/rasters/data/xt_SENTINEL2B_20180621-111349-432_L2A_T30TWT_D_V1-8_RVBPIR_cog.tif"

In [2]:
# Generate the HDF5 file (check out the datacube_formats for more information on the HDF5 format)
hdf5_file = "./sample_data/data_cube/example.h5"  # change this path if needed
generate_hdf5_file(hdf5_file, cog_path) 

In [3]:
# read the entire raster (for comparison)
with rasterio.open(cog_path) as src:
    %time data = src.read(1)

CPU times: user 87.2 ms, sys: 45.2 ms, total: 132 ms
Wall time: 131 ms


In [4]:
# read chunk by chunk
x_chunk_size = 1000  # number of columns per chunk
y_chunk_size = 1000  # number of rows per chunk

with rasterio.open(cog_path) as src:
    width = src.width
    height = src.height
    for start_row in range(0, height, y_chunk_size):  # iterate over rows
        end_row = min(start_row + y_chunk_size, height)
        for start_col in range(0, width, x_chunk_size):  # iterate over columns
            end_col = min(start_col + x_chunk_size, width)
            window = Window(start_col, start_row, end_col - start_col, end_row - start_row)
            %time data = src.read(1, window=window)  # read the 1st band
            print(data.shape)

CPU times: user 17.8 ms, sys: 8.33 ms, total: 26.2 ms
Wall time: 25.4 ms
(1000, 1000)
CPU times: user 13.7 ms, sys: 11.9 ms, total: 25.6 ms
Wall time: 25.4 ms
(1000, 1000)
CPU times: user 7.7 ms, sys: 1.96 ms, total: 9.66 ms
Wall time: 9.67 ms
(1000, 410)
CPU times: user 14.7 ms, sys: 10.5 ms, total: 25.2 ms
Wall time: 25.2 ms
(1000, 1000)
CPU times: user 16.7 ms, sys: 8.3 ms, total: 25 ms
Wall time: 25 ms
(1000, 1000)
CPU times: user 8.21 ms, sys: 1.8 ms, total: 10 ms
Wall time: 10 ms
(1000, 410)
CPU times: user 1.26 ms, sys: 36 μs, total: 1.29 ms
Wall time: 1.3 ms
(80, 1000)
CPU times: user 1.12 ms, sys: 845 μs, total: 1.97 ms
Wall time: 1.97 ms
(80, 1000)
CPU times: user 0 ns, sys: 1.08 ms, total: 1.08 ms
Wall time: 1.08 ms
(80, 410)


But there is an easier and more optimized way of dealing with rasters by chunks. COGs have an intern chunk size used when writing them. It is easier and usually more efficient to use this chunk size instead, using `src.block_windows`. In this code `ji` is the current chunk's coordinates, relative to the chunk grid. The coordinates of the first pixel of the chunk are given by the widow's column and row offset.

In [5]:
with rasterio.open(cog_path) as src:
    for ji, window in src.block_windows(1):
        print(f"Chunk's coordinates: {ji}")
        print(window)
        data = src.read(window=window)
        print(data.shape, "\n")

Chunk's coordinates: (0, 0)
Window(col_off=0, row_off=0, width=256, height=256)
(4, 256, 256) 

Chunk's coordinates: (0, 1)
Window(col_off=256, row_off=0, width=256, height=256)
(4, 256, 256) 

Chunk's coordinates: (0, 2)
Window(col_off=512, row_off=0, width=256, height=256)
(4, 256, 256) 

Chunk's coordinates: (0, 3)
Window(col_off=768, row_off=0, width=256, height=256)
(4, 256, 256) 

Chunk's coordinates: (0, 4)
Window(col_off=1024, row_off=0, width=256, height=256)
(4, 256, 256) 

Chunk's coordinates: (0, 5)
Window(col_off=1280, row_off=0, width=256, height=256)
(4, 256, 256) 

Chunk's coordinates: (0, 6)
Window(col_off=1536, row_off=0, width=256, height=256)
(4, 256, 256) 

Chunk's coordinates: (0, 7)
Window(col_off=1792, row_off=0, width=256, height=256)
(4, 256, 256) 

Chunk's coordinates: (0, 8)
Window(col_off=2048, row_off=0, width=256, height=256)
(4, 256, 256) 

Chunk's coordinates: (0, 9)
Window(col_off=2304, row_off=0, width=106, height=256)
(4, 256, 106) 

Chunk's coordina

### Using rioxarray

Rasterio opens raster as numpy arrays and processes them sequentially. Instead, it is possible to use `rioxarray`, a library to open rasters as xarrays:

In [6]:
import rioxarray as rxr

xds = rxr.open_rasterio(cog_path, chunks=True)
xds

<xarray.DataArray (band: 4, y: 2080, x: 2410)> Size: 80MB
dask.array<open_rasterio-e740e86a22526fc8a2f3362c1fd70008<this-array>, shape=(4, 2080, 2410), dtype=float32, chunksize=(1, 2080, 2410), chunktype=numpy.ndarray>
Coordinates:
  * band         (band) int64 32B 1 2 3 4
  * x            (x) float64 19kB 5.013e+05 5.013e+05 ... 5.253e+05 5.253e+05
  * y            (y) float64 17kB 5.279e+06 5.279e+06 ... 5.258e+06 5.258e+06
    spatial_ref  int64 8B 0
Attributes:
    AREA_OR_POINT:  Area
    scale_factor:   1.0
    add_offset:     0.0

The `rioxarray` [official documentation](https://corteva.github.io/rioxarray/stable/examples/read-locks.html#Chunking) details the chunking process and how to optimize chunk management. This also allows the use of `dask` to process chunks in parallel. Once again, the official documentation has details on how to use dask with rioxarray: https://corteva.github.io/rioxarray/stable/examples/dask_read_write.html.

## Vector data

Vector data allows two types of optimizations:

- streaming data chunk by chunk (similar to raster files)
- filtering input data before loading it in memory

These optimizations are not necesarily available for all vector formats. Geoparquet is the most suitable format for these uses: it can be easily streamed and filtered. 

### Streaming

First, let's try reading data chunk by chunk. This can be easily done using `pyarrow`. To illustrate this example, we'll re-use the `landuse.parquet` file generated in the [vector formats tutorial](./vector_data_formats.ipynb) and count the number of forest polygons in each chunk (or batch).

In [7]:
import pyarrow.parquet as pq
file_path = "./sample_data/vector/departement-31/landuse.parquet"

parquet_file = pq.ParquetFile(file_path)

for n, batch in enumerate(parquet_file.iter_batches(batch_size=1000)):  # batch_size: number of rows to read
    print(f"Batch number {n}, n_rows: {batch.num_rows}")
    column=batch.column("type").to_pandas()  # select the column 'type' and transform the column to a pandas series
    n_forest = len(column[column == "forest"].index)  # count the occurrences of 'forest' 
    print(f"{n_forest} occurences of 'forest'\n")

Batch number 0, n_rows: 1000
316 occurences of 'forest'

Batch number 1, n_rows: 1000
273 occurences of 'forest'

Batch number 2, n_rows: 1000
515 occurences of 'forest'

Batch number 3, n_rows: 1000
245 occurences of 'forest'

Batch number 4, n_rows: 1000
286 occurences of 'forest'

Batch number 5, n_rows: 1000
421 occurences of 'forest'

Batch number 6, n_rows: 1000
423 occurences of 'forest'

Batch number 7, n_rows: 1000
437 occurences of 'forest'

Batch number 8, n_rows: 1000
184 occurences of 'forest'

Batch number 9, n_rows: 1000
75 occurences of 'forest'

Batch number 10, n_rows: 1000
28 occurences of 'forest'

Batch number 11, n_rows: 1000
11 occurences of 'forest'

Batch number 12, n_rows: 1000
11 occurences of 'forest'

Batch number 13, n_rows: 1000
15 occurences of 'forest'

Batch number 14, n_rows: 1000
26 occurences of 'forest'

Batch number 15, n_rows: 297
49 occurences of 'forest'



## Datacube

Raster (and, in a less significant way vector) formats can be optimized for cloud computing, but may require some non-trivial work. However, most datacube formats are natively cloud-optimized (or can easily be optimized for cloud usage), like zarr. Let's reuse the `.zarr` file generated in the [data cube formats tutorial](./datacube_formats.ipynb). As explained in that tutorial, `.zarr` files are natively split into chunks to facilitate chunking mechanisms. Another optimization is lazy loading and computing. Lazy loading means that the data is not loaded into memory until needed; only the metadata is stored. This allows the user to define operations that, similarly, are not computed until required. The loading and computing steps are then optimized and executed at the end, improving ressource usage. Here is an example:

In [8]:
import hvplot.xarray
import xarray as xr
zarr_path = "./sample_data/data_cube/example_from_xarray.zarr"

da = xr.open_dataset(zarr_path)["data"]
da.variable

/home/ecadaux/Documents/work/pluto-tuto/data_type/pluto_tuto/lib/python3.12/site-packages/xarray/backends/api.py:652: RuntimeWarning: 'netcdf4' fails while guessing
  engine = plugins.guess_engine(filename_or_obj)
/home/ecadaux/Documents/work/pluto-tuto/data_type/pluto_tuto/lib/python3.12/site-packages/xarray/backends/api.py:652: RuntimeWarning: 'scipy' fails while guessing
  engine = plugins.guess_engine(filename_or_obj)


<xarray.Variable (time: 10, lat: 1000, lon: 1000)> Size: 80MB
[10000000 values with dtype=float64]

The actual values of the array aren't loaded. One way of explicitly loading them into memory is using the `.load()` method:

In [9]:
da.load()
da.variable

<xarray.Variable (time: 10, lat: 1000, lon: 1000)> Size: 80MB
array([[[2.18448228e-01, 1.77242407e-01, 2.33341841e-01, ...,
         6.57756970e-01, 9.94811207e-02, 1.19356576e-01],
        [2.39467312e-01, 9.17736540e-02, 9.02144187e-01, ...,
         2.52061223e-02, 7.53525289e-01, 6.26194626e-02],
        [5.78309337e-01, 6.43915144e-01, 9.50565304e-01, ...,
         9.28949251e-02, 3.57590228e-01, 2.04823118e-01],
        ...,
        [5.03667111e-01, 7.54269344e-01, 8.24519391e-01, ...,
         1.09653237e-01, 2.73826898e-02, 2.73352545e-01],
        [4.11992847e-02, 4.41216625e-01, 9.61109269e-01, ...,
         6.76729535e-01, 2.91302801e-01, 7.73114982e-01],
        [3.43041668e-01, 1.20612258e-01, 5.21666067e-01, ...,
         1.68927151e-01, 6.98497429e-01, 2.22854006e-01]],

       [[5.35333673e-01, 5.01983535e-01, 5.52297300e-01, ...,
         8.64630501e-01, 8.77514542e-01, 4.20768771e-02],
        [7.11241503e-01, 6.66998721e-02, 8.60773527e-01, ...,
         6.05696959e-01, 7.74040591e-01, 9.75836502e-01],
        [8.41105094e-02, 4.56205279e-02, 9.82640008e-01, ...,
         5.83621360e-02, 5.44525578e-01, 5.98770606e-01],
...
        [3.28544592e-01, 3.87033096e-01, 8.67012328e-01, ...,
         7.43674013e-01, 7.90014269e-01, 3.89947925e-02],
        [2.05081013e-01, 6.16911549e-01, 4.56712823e-01, ...,
         7.95287118e-01, 4.46309842e-01, 4.35709169e-01],
        [2.38760504e-01, 9.82719446e-01, 5.59488249e-01, ...,
         7.01931610e-02, 2.01297375e-01, 8.71147819e-02]],

       [[6.78375257e-01, 7.51888830e-01, 4.93826845e-01, ...,
         1.41665720e-01, 2.00191777e-01, 6.73765282e-02],
        [5.74372667e-02, 7.98653199e-01, 9.68140427e-01, ...,
         7.31659314e-01, 3.75235722e-01, 5.75888345e-01],
        [6.61670795e-01, 6.59940239e-01, 9.48021593e-02, ...,
         8.22612536e-01, 3.85461299e-01, 2.97097138e-01],
        ...,
        [1.59427564e-01, 5.33449663e-01, 3.63135655e-01, ...,
         8.18862455e-01, 6.32256476e-04, 1.78639742e-01],
        [6.21602394e-01, 1.81945922e-01, 6.84754812e-01, ...,
         7.70042344e-01, 2.02440931e-01, 1.56071414e-01],
        [1.63580076e-01, 7.10248503e-01, 1.25715940e-01, ...,
         9.74522596e-01, 1.42963532e-01, 1.55423297e-01]]])

Let's reopen it to reset it to lazy loaded and test lazy computing. For this, let's multiply the array by 10 and compute its mean over all time periods:

In [10]:
da = xr.open_dataset(zarr_path, chunks="auto", engine="zarr")["data"]

print("Lazy loaded array:")  # lazy loaded
print(da, "\n\n")

scaled_data = da * 10
mean_data = scaled_data.mean(dim="time")  # lazy computed
print("Lazy computed array:")
print(mean_data, "\n\n")


result = mean_data.compute()  # load in memory and compute
print("Loaded and computed:")
print(result)

Lazy loaded array:
<xarray.DataArray 'data' (time: 10, lat: 1000, lon: 1000)> Size: 80MB
dask.array<open_dataset-data, shape=(10, 1000, 1000), dtype=float64, chunksize=(10, 1000, 1000), chunktype=numpy.ndarray>
Coordinates:
  * lat      (lat) int64 8kB 0 1 2 3 4 5 6 7 ... 992 993 994 995 996 997 998 999
  * lon      (lon) int64 8kB 0 1 2 3 4 5 6 7 ... 992 993 994 995 996 997 998 999
  * time     (time) int64 80B 0 1 2 3 4 5 6 7 8 9 


Lazy computed array:
<xarray.DataArray 'data' (lat: 1000, lon: 1000)> Size: 8MB
dask.array<mean_agg-aggregate, shape=(1000, 1000), dtype=float64, chunksize=(1000, 1000), chunktype=numpy.ndarray>
Coordinates:
  * lat      (lat) int64 8kB 0 1 2 3 4 5 6 7 ... 992 993 994 995 996 997 998 999
  * lon      (lon) int64 8kB 0 1 2 3 4 5 6 7 ... 992 993 994 995 996 997 998 999 


Loaded and computed:
<xarray.DataArray 'data' (lat: 1000, lon: 1000)> Size: 8MB
array([[4.11661461, 4.37367803, 3.70126257, ..., 5.405144  , 3.90996162,
        4.11714764],
       [4.4194

As explained in the rasters section, an efficient optimization method is to run chunks computation in parallel, using dask for example. And `xarray` uses dask to handle data arrays, making it easier to assemble these methods.

## Conclusion

There are many optimization techniques available for all the data types and formats presented, usable both on local and cloud architectures. It is important to consider which format and what processing methods can and should be used, depending on the architecture and ressources available, as well as the data volume. Generally, a modern and optimized code should be considered, but over a  small dataset, using parallel processing and chunking data may introduce unnecessary complexity and might be inefficient.